# Arbitrage and bond markets

In [1]:
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)

## Bootstrapping

In [18]:
A = [  98.04,      0, 0.5, 100]
B = [  96.12,      0, 1.0, 100]
C = [ 971.16,   0.02, 1.5, 1000]
D = [1019.04,   0.05, 2.0, 1000]
E = [1000.00,   0.04, 2.5, 1000]
F = [1028.01,   0.05, 3.0, 1000]
bonds = np.row_stack((A, B, C, D, E, F))

In [19]:
df = pd.DataFrame(bonds,
        columns = ['price', 'coupon_rate', 'maturity','face'],
        index=['A','B','C','D','E','F'])
df['period'] = df.maturity*2
df=df.reset_index()
df.index = df.index+1
df.columns = ['bond', 'price', 'coupon_rate', 'maturity','face', 'period']
df

,bond,price,coupon_rate,maturity,face,period
1,A,98.04,0.00,0.5,100.0,1.0
2,B,96.12,0.00,1.0,100.0,2.0
3,C,971.16,0.02,1.5,1000.0,3.0
4,D,1019.04,0.05,2.0,1000.0,4.0
5,E,1000.00,0.04,2.5,1000.0,5.0
6,F,1028.01,0.05,3.0,1000.0,6.0


### Functions to generalize bootstrapping and pricing

In [20]:
def bootstrap(d, nop):
    '''
    d: a dataset with prices, face value, coupon_rate, period of maturity
    nop: compounding period (ie., 1 = annual, 2=semiannual)
    '''
    spot_rates = []
    for t in d.index:
        p      = d.loc[t,'price']
        face   = d.loc[t,'face']
        cr     = d.loc[t,'coupon_rate']
        period = int(d.loc[t,'period'])
        coupon = face*cr/nop
        cf_maturity = face + coupon

        pv_coupons = 0.0
        for i in range(period-1):
            pv_coupons = pv_coupons + coupon / (1+spot_rates[i])**(i+1)
        x = p - pv_coupons
        z = (cf_maturity / x )**(1/period) - 1
        spot_rates.append(z)
    return spot_rates
spot_curve = bootstrap(df,2)
spot_curve

[0.01999184006527943,
 0.01998343560350757,
 0.020000556539538028,
 0.019999887297390107,
 0.020000167501260657,
 0.019999642143768703]

In [21]:
def spot_pricer(cfs, spot):
    '''
    cfs:  list of cash flows at each period
    spot: list of spot rates at each period
    cfs should be the same length as spot
    '''
    cfs = np.array(cfs)
    discount_factors = np.array([1/(1+spot[i])**(i+1) for i in range(len(spot))])
    return cfs @ discount_factors
spot_pricer([0,40,0,40,0,1040], spot_curve)

998.8940148304082

In [22]:
# Check that the prices are consistent with the spot curve

# pick a bond
BOND_ID = 'A'
bond_dict = {'A': 1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6}
def cf_generator(bond_id):
    i = bond_dict[bond_id]
    nop = 2
    face   = df.loc[i,'face']
    cr     = df.loc[i,'coupon_rate']
    period = int(df.loc[i,'period'])
    coupon = face*cr/nop
    cf_maturity = face + coupon
    cfs =  np.zeros(len(spot_curve))
    for i in range(period-1):
        cfs[i] = coupon
    cfs[period-1] = cf_maturity
    print('The bond cash flows for bond ' + bond_id + ' are: ', cfs)
    return cfs
cfs = cf_generator(BOND_ID)

p = spot_pricer(cfs.tolist(),spot_curve)
print('The fair price for ' + BOND_ID + f' is: ${p:,.2f}')


The bond cash flows for bond A are:  [100.   0.   0.   0.   0.   0.]
The fair price for A is: $98.04


### Arbitrage and replicating portfolio


Example #1 (2-period bond)

In [23]:
spot_pricer([0,40,0,40,0,1040], spot_curve)

998.8940148304082

Example #2 (4-period bond)

In [24]:
# Target bond cash-flows
cfs = np.array([0, 40, 0, 40, 0, 1040])
cfs

array([   0,   40,    0,   40,    0, 1040])

In [25]:
# matrix of cash flows for bonds A-D
CF = np.zeros((len(df), len(df)))
for j, id in enumerate(['A','B','C','D','E','F']):
    print(j)
    print(id)
    CF[j] = cf_generator(id)
CF

0
A
The bond cash flows for bond A are:  [100.   0.   0.   0.   0.   0.]
1
B
The bond cash flows for bond B are:  [  0. 100.   0.   0.   0.   0.]
2
C
The bond cash flows for bond C are:  [  10.   10. 1010.    0.    0.    0.]
3
D
The bond cash flows for bond D are:  [  25.   25.   25. 1025.    0.    0.]
4
E
The bond cash flows for bond E are:  [  20.   20.   20.   20. 1020.    0.]
5
F
The bond cash flows for bond F are:  [  25.   25.   25.   25.   25. 1025.]


array([[ 100.,    0.,    0.,    0.,    0.,    0.],
       [   0.,  100.,    0.,    0.,    0.,    0.],
       [  10.,   10., 1010.,    0.,    0.,    0.],
       [  25.,   25.,   25., 1025.,    0.,    0.],
       [  20.,   20.,   20.,   20., 1020.,    0.],
       [  25.,   25.,   25.,   25.,   25., 1025.]])

In [26]:
# Solve the system of equations
portfolio = cfs @ np.linalg.pinv(CF)
portfolio

array([-0.24987669,  0.15012331, -0.02498767,  0.01476245, -0.02486848,
        1.01463415])

In [27]:
# Check that replicating portfolio delivers same cash flows
repl_cfs = portfolio @ CF
repl_cfs.astype(int)

array([   0,   40,    0,   39,    0, 1039])

In [28]:
# Price of replicating portfolio
repl_price = portfolio @ df.price
repl_price

998.8940148304073

In [29]:
profit = repl_price-1010
profit

-11.105985169592714

In [30]:
A = [ 97.09,      0, 0.5, 100]
B = [ 94.26,      0, 1.0, 100]
C = [943.43,   0.02, 1.5, 1000]
D = [981.41,   0.05, 2.0, 1000]
E = [954.20,   0.04, 2.5, 1000]
F = [972.91,   0.05, 3.0, 1000]
G = [906.55,   0.03, 3.5, 1000]
H = [964.90,   0.05, 4.0, 1000]
bonds = np.row_stack((A, B, C, D, E, F, G, H))

In [31]:
df = pd.DataFrame(bonds,
        columns = ['price', 'coupon_rate', 'maturity','face'],
        index=['A','B','C','D','E','F','G','H'])
df['period'] = df.maturity*2
df=df.reset_index()
df.index = df.index+1
df.columns = ['bond', 'price', 'coupon_rate', 'maturity','face', 'period']
df

,bond,price,coupon_rate,maturity,face,period
1,A,97.09,0.00,0.5,100.0,1.0
2,B,94.26,0.00,1.0,100.0,2.0
3,C,943.43,0.02,1.5,1000.0,3.0
4,D,981.41,0.05,2.0,1000.0,4.0
5,E,954.20,0.04,2.5,1000.0,5.0
6,F,972.91,0.05,3.0,1000.0,6.0
7,G,906.55,0.03,3.5,1000.0,7.0
8,H,964.90,0.05,4.0,1000.0,8.0


### Functions to generalize bootstrapping and pricing

In [32]:
def bootstrap(d, nop):
    '''
    d: a dataset with prices, face value, coupon_rate, period of maturity
    nop: compounding period (ie., 1 = annual, 2=semiannual)
    '''
    spot_rates = []
    for t in d.index:
        p      = d.loc[t,'price']
        face   = d.loc[t,'face']
        cr     = d.loc[t,'coupon_rate']
        period = int(d.loc[t,'period'])
        coupon = face*cr/nop
        cf_maturity = face + coupon

        pv_coupons = 0.0
        for i in range(period-1):
            pv_coupons = pv_coupons + coupon / (1+spot_rates[i])**(i+1)
        x = p - pv_coupons
        z = (cf_maturity / x )**(1/period) - 1
        spot_rates.append(z)
    return spot_rates
spot_curve = bootstrap(df,2)
spot_curve

[0.02997219075084967,
 0.029997764907275126,
 0.0299992853016684,
 0.030001502317938078,
 0.030000812108414543,
 0.030000926589999644,
 0.029999294362616613,
 0.030000336401881267]

In [33]:
def spot_pricer(cfs, spot):
    '''
    cfs:  list of cash flows at each period
    spot: list of spot rates at each period
    cfs should be the same length as spot
    '''
    cfs = np.array(cfs)
    discount_factors = np.array([1/(1+spot[i])**(i+1) for i in range(len(spot))])
    return cfs @ discount_factors
spot_pricer([50,50,50,50,50,50,50,1050], spot_curve)

1140.3928282752777

In [34]:
# Check that the prices are consistent with the spot curve

# pick a bond
BOND_ID = 'A'
bond_dict = {'A': 1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8}
def cf_generator(bond_id):
    i = bond_dict[bond_id]
    nop = 2
    face   = df.loc[i,'face']
    cr     = df.loc[i,'coupon_rate']
    period = int(df.loc[i,'period'])
    coupon = face*cr/nop
    cf_maturity = face + coupon
    cfs =  np.zeros(len(spot_curve))
    for i in range(period-1):
        cfs[i] = coupon
    cfs[period-1] = cf_maturity
    print('The bond cash flows for bond ' + bond_id + ' are: ', cfs)
    return cfs
cfs = cf_generator(BOND_ID)

p = spot_pricer(cfs.tolist(),spot_curve)
print('The fair price for ' + BOND_ID + f' is: ${p:,.2f}')


The bond cash flows for bond A are:  [100.   0.   0.   0.   0.   0.   0.   0.]
The fair price for A is: $97.09


### Arbitrage and replicating portfolio


Example #1 (2-period bond)

In [35]:
spot_pricer([50,50,50,50,50,50,50,1050], spot_curve)

1140.3928282752777

Example #2 (4-period bond)

In [36]:
# Target bond cash-flows
cfs = np.array([50,50,50,50,50,50,50,1050])
cfs

array([  50,   50,   50,   50,   50,   50,   50, 1050])

In [37]:
# matrix of cash flows for bonds A-D
CF = np.zeros((len(df), len(df)))
for j, id in enumerate(['A','B','C','D','E','F','G','H']):
    print(j)
    print(id)
    CF[j] = cf_generator(id)
CF

0
A
The bond cash flows for bond A are:  [100.   0.   0.   0.   0.   0.   0.   0.]
1
B
The bond cash flows for bond B are:  [  0. 100.   0.   0.   0.   0.   0.   0.]
2
C
The bond cash flows for bond C are:  [  10.   10. 1010.    0.    0.    0.    0.    0.]
3
D
The bond cash flows for bond D are:  [  25.   25.   25. 1025.    0.    0.    0.    0.]
4
E
The bond cash flows for bond E are:  [  20.   20.   20.   20. 1020.    0.    0.    0.]
5
F
The bond cash flows for bond F are:  [  25.   25.   25.   25.   25. 1025.    0.    0.]
6
G
The bond cash flows for bond G are:  [  15.   15.   15.   15.   15.   15. 1015.    0.]
7
H
The bond cash flows for bond H are:  [  25.   25.   25.   25.   25.   25.   25. 1025.]


array([[ 100.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,  100.,    0.,    0.,    0.,    0.,    0.,    0.],
       [  10.,   10., 1010.,    0.,    0.,    0.,    0.,    0.],
       [  25.,   25.,   25., 1025.,    0.,    0.,    0.,    0.],
       [  20.,   20.,   20.,   20., 1020.,    0.,    0.,    0.],
       [  25.,   25.,   25.,   25.,   25., 1025.,    0.,    0.],
       [  15.,   15.,   15.,   15.,   15.,   15., 1015.,    0.],
       [  25.,   25.,   25.,   25.,   25.,   25.,   25., 1025.]])

In [38]:
# Solve the system of equations
portfolio = cfs @ np.linalg.pinv(CF)
portfolio

array([0.22201424, 0.22201424, 0.02220142, 0.02242344, 0.02298402,
       0.0234437 , 0.0240298 , 1.02439024])

In [39]:
# Check that replicating portfolio delivers same cash flows
repl_cfs = portfolio @ CF
repl_cfs.astype(int)

array([  50,   50,   50,   50,   50,   49,   49, 1049])

In [40]:
# Price of replicating portfolio
repl_price = portfolio @ df.price
repl_price

1140.3928282752777

In [74]:
profit = repl_price-1010
profit

130.39282827527768

In [ ]:
#jonathans#

In [79]:
A = [ 97.09,      0, 0.5, 100]
B = [ 94.26,      0, 1.0, 100]
C = [943.43,   0.02, 1.5, 1000]
D = [981.41,   0.05, 2.0, 1000]
E = [954.20,   0.04, 2.5, 1000]
F = [972.91,   0.05, 3.0, 1000]
bonds = np.row_stack((A, B, C, D, E, F))

In [80]:
df = pd.DataFrame(bonds,
        columns = ['price', 'coupon_rate', 'maturity','face'],
        index=['A','B','C','D','E','F'])
df['period'] = df.maturity*2
df=df.reset_index()
df.index = df.index+1
df.columns = ['bond', 'price', 'coupon_rate', 'maturity','face', 'period']
df

,bond,price,coupon_rate,maturity,face,period
1,A,97.09,0.00,0.5,100.0,1.0
2,B,94.26,0.00,1.0,100.0,2.0
3,C,943.43,0.02,1.5,1000.0,3.0
4,D,981.41,0.05,2.0,1000.0,4.0
5,E,954.20,0.04,2.5,1000.0,5.0
6,F,972.91,0.05,3.0,1000.0,6.0


### Functions to generalize bootstrapping and pricing

In [81]:
def bootstrap(d, nop):
    '''
    d: a dataset with prices, face value, coupon_rate, period of maturity
    nop: compounding period (ie., 1 = annual, 2=semiannual)
    '''
    spot_rates = []
    for t in d.index:
        p      = d.loc[t,'price']
        face   = d.loc[t,'face']
        cr     = d.loc[t,'coupon_rate']
        period = int(d.loc[t,'period'])
        coupon = face*cr/nop
        cf_maturity = face + coupon

        pv_coupons = 0.0
        for i in range(period-1):
            pv_coupons = pv_coupons + coupon / (1+spot_rates[i])**(i+1)
        x = p - pv_coupons
        z = (cf_maturity / x )**(1/period) - 1
        spot_rates.append(z)
    return spot_rates
spot_curve = bootstrap(df,2)
spot_curve

[0.02997219075084967,
 0.029997764907275126,
 0.0299992853016684,
 0.030001502317938078,
 0.030000812108414543,
 0.030000926589999644]

In [82]:
def spot_pricer(cfs, spot):
    '''
    cfs:  list of cash flows at each period
    spot: list of spot rates at each period
    cfs should be the same length as spot
    '''
    cfs = np.array(cfs)
    discount_factors = np.array([1/(1+spot[i])**(i+1) for i in range(len(spot))])
    return cfs @ discount_factors
spot_pricer([0,70,0,70,0,1070], spot_curve)

944.222200305792

In [83]:
# Check that the prices are consistent with the spot curve

# pick a bond
BOND_ID = 'A'
bond_dict = {'A': 1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6}
def cf_generator(bond_id):
    i = bond_dict[bond_id]
    nop = 2
    face   = df.loc[i,'face']
    cr     = df.loc[i,'coupon_rate']
    period = int(df.loc[i,'period'])
    coupon = face*cr/nop
    cf_maturity = face + coupon
    cfs =  np.zeros(len(spot_curve))
    for i in range(period-1):
        cfs[i] = coupon
    cfs[period-1] = cf_maturity
    print('The bond cash flows for bond ' + bond_id + ' are: ', cfs)
    return cfs
cfs = cf_generator(BOND_ID)

p = spot_pricer(cfs.tolist(),spot_curve)
print('The fair price for ' + BOND_ID + f' is: ${p:,.2f}')


The bond cash flows for bond A are:  [100.   0.   0.   0.   0.   0.]
The fair price for A is: $97.09


### Arbitrage and replicating portfolio


Example #1 (2-period bond)

In [92]:
spot_pricer([0,70,0,70,0,1070], spot_curve)

1024.2790483228048

Example #2 (4-period bond)

In [100]:
# Target bond cash-flows
cfs = np.array([0,70,0,70,0,1070])
cfs

array([   0,   70,    0,   70,    0, 1070])

In [101]:
# matrix of cash flows for bonds A-D
CF = np.zeros((len(df), len(df)))
for j, id in enumerate(['A','B','C','D','E','F']):
    print(j)
    print(id)
    CF[j] = cf_generator(id)
CF

0
A
The bond cash flows for bond A are:  [100.   0.   0.   0.   0.   0.]
1
B
The bond cash flows for bond B are:  [  0. 100.   0.   0.   0.   0.]
2
C
The bond cash flows for bond C are:  [  10.   10. 1010.    0.    0.    0.]
3
D
The bond cash flows for bond D are:  [  25.   25.   25. 1025.    0.    0.]
4
E
The bond cash flows for bond E are:  [  20.   20.   20.   20. 1020.    0.]
5
F
The bond cash flows for bond F are:  [  25.   25.   25.   25.   25. 1025.]


array([[ 100.,    0.,    0.,    0.,    0.,    0.],
       [   0.,  100.,    0.,    0.,    0.,    0.],
       [  10.,   10., 1010.,    0.,    0.,    0.],
       [  25.,   25.,   25., 1025.,    0.,    0.],
       [  20.,   20.,   20.,   20., 1020.,    0.],
       [  25.,   25.,   25.,   25.,   25., 1025.]])

In [102]:
# Solve the system of equations
portfolio = cfs @ np.linalg.pinv(CF)
portfolio

array([-0.26405066,  0.43594934, -0.02640507,  0.04333088, -0.02558584,
        1.04390244])

In [103]:
# Check that replicating portfolio delivers same cash flows
repl_cfs = portfolio @ CF
repl_cfs.astype(int)

array([   0,   70,    0,   69,    0, 1069])

In [104]:
# Price of replicating portfolio
repl_price = portfolio @ df.price
repl_price

1024.2790483228023

In [105]:
profit = repl_price-1050
profit

-25.720951677197718